In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "cat"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
                               





In [7]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1(children='Zuzana Martins', 
       style={'textAlign': 'left', 
          'color': '#6EA2D6'}))),
    html.A([
        
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'textAlign': 'center',
                'height' : '20%',
                'width' : '20%',
            })
    ], href='https://www.snhu.edu'),     #image is clickable   
            
    html.Hr(),
    html.Div(
        dcc.RadioItems( #radio items for easy filtering
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MOWR'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DROIT'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block'}
        )  
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        hidden_columns=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

app

In [11]:
#############################################
# Interaction Between Components / Controller
#############################################
  
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
#queries for Mongo to update table    
    WR = shelter.read( { "$or": [ {"breed" : "Labrador Retriever Mix"}, 
                                              {"breed" : "Chesapeake Bay Retriever"}, 
                                              {"breed" : "Newfoundland"} ] }, 
                                     {"sex_upon_outcome" : "Intact Female"}, 
                                     {"age_upon_outcome_in_weeks" :{"$lt" : 156, "$gte" : 26}}, 
                                     {"animal_id" : 1}).sort({"age_upon_outcome_in_weeks" : 1})                                                            
    MOWR = shelter.read( { "$or": [ {"breed" : "German Shepherd"}, 
                                              {"breed" : "Alaskan Malamute"}, 
                                              {"breed" : "Old English Sheepdog"},
                                              {"breed" : "Siberian Husky"}, 
                                              {"breed" : "Rottweiler"} ] }, 
                                     {"sex_upon_outcome" : "Intact Male"}, 
                                     {"age_upon_outcome_in_weeks" :{"$lt" : 156, "$gte" : 26}}, 
                                     {"animal_id" : 1}).sort({"age_upon_outcome_in_weeks" : 1})                                                            
    DROIT = shelter.read( { "$or": [ {"breed" : "Doberman Pinscher"}, 
                                              {"breed" : "German Shepherd"}, 
                                              {"breed" : "Bloodhound"},
                                              {"breed" : "Rottweiler"},
                                              {"breed" : "Golden Retriever"} ] }, 
                                     {"sex_upon_outcome" : "Intact Male"}, 
                                     {"age_upon_outcome_in_weeks" :{"$lt" : 300, "$gte" : 26}}, 
                                     {"animal_id" : 1}).sort({"age_upon_outcome_in_weeks" : 1})    
### FIX ME Add code to filter interactive data table with MongoDB queries ^^^^

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
          
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#graph/pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):


#map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):


app